In [4]:

import firebase_admin
from firebase_admin import credentials, firestore


service_account_path = "/Users/narayanpowderly/Documents/atopile-workspace/package-registry-backend/atopile-880ca67acfe2.json"

if not firebase_admin._apps:  # Check if already initialized to prevent reinitialization
    cred = credentials.Certificate(service_account_path)
    firebase_admin.initialize_app(cred)

db = firestore.client()
_components_db = db.collection(
    "components"
)  

In [16]:
import math

def get_resistor_or_capacitor(req_json, firestore_client):
    # Get the component requirements
    min_value = req_json.get("min_value")
    max_value = req_json.get("max_value")
    package = req_json.get("package", "").lstrip('C').lstrip('R')  # Strip leading 'R' if package is specified
    voltage = req_json.get("voltage")
    voltage_factor_of_safety = req_json.get("voltage_factor_of_safety", 2)
    dielectric = req_json.get("dielectric")

    # Query and filter
    query = firestore_client.collection("components").where("type", "==", req_json.get("type"))
    results = query.stream()
    components = [doc.to_dict() for doc in results]
    filtered_components = []
    for component in components:
        try:
            valid = True
            component_min_value = component.get("min_value")
            component_max_value = component.get("max_value")

            # Adjusted conditions
            if (component_min_value is None or math.isnan(component_min_value) or
                min_value is not None and component_min_value < min_value):
                valid = False
            if (component_max_value is None or math.isnan(component_max_value) or
                max_value is not None and component_max_value > max_value):
                valid = False
            if package and component.get("Package") != package:
                valid = False
            if component.get("Stock", 0) <= 100:  # Check stock only if other conditions are met
                valid = False
            if voltage and component.get("voltage") < voltage * voltage_factor_of_safety:
                valid = False
            if dielectric and component.get("dielectric") != dielectric:
                valid = False
            if valid:
                filtered_components.append(component)
        except (TypeError, ValueError, KeyError):
            continue

    # Select the best component
    basic_components = [component for component in filtered_components if component.get("basic-extended") == "Basic"]
    if basic_components:
        best_component = min(basic_components, key=lambda component: component["Price (USD)"])
    else:
        best_component = min(filtered_components, key=lambda component: component["Price (USD)"]) if filtered_components else None

    return {"Best Component": best_component}


In [20]:
payload = {
    "type": "Capacitor",
    "min_value": 0.00000176,
    "max_value": 0.00000264,
    "voltage": 50,
    # "dielectric": "X7R",
    # "package": "0402"
}

get_resistor_or_capacitor(payload, db)

{'Best Component': {'value': 2.2,
  'type': 'Capacitor',
  'basic-extended': 'Extended Part',
  'Description': 'Multilayer Ceramic Capacitors MLCC - SMD/SMT 2.2uF 100V 1210 RoHS',
  'Price (USD)': nan,
  'Stock': nan,
  'code': nan,
  'Datasheet': nan,
  'Type': nan,
  'Package': '1210',
  'voltage': 100.0,
  'min_value': 1.98e-06,
  'MFR.Part #': 'CL32B225KCJSNNE',
  'tolerance': 10.0,
  'value_in_ohms': nan,
  'unit': 'uF',
  'LCSC Part #': 'C55151',
  'Comment': '2.2uF(225) ±10% 100V X7R',
  'value_in_farads': 2.2e-06,
  'Category': 'Multilayer Ceramic Capacitors MLCC - SMD/SMT',
  'dielectric': 'X7R',
  'max_value': 2.42e-06}}

In [28]:
import json
import math

def get_mosfet(req_json, firestore_client):
    try:
        # Retrieve the request parameters
        polarity = req_json.get("polarity")
        ds_voltage = req_json.get("drain_source_voltage_V")
        current = req_json.get("current_A")
        on_resistance = req_json.get("on_resistance_ohms")
        power_dissipation = req_json.get("power_dissipation_W")
        safety_factor_current = req_json.get("safety_factor", 1.5)
        safety_factor_voltage = req_json.get("safety_factor", 1.5)
        package = req_json.get("Package")

        if not polarity:
            raise ValueError("Polarity is required.")

        # Query the collection
        query = firestore_client.collection("mosfets").where("polarity", "==", polarity)
        results = query.stream()
        components = [doc.to_dict() for doc in results]
        print(len(components))
        # Filter components based on provided criteria
        filtered_components = []
        for component in components:
            valid = True
            if ds_voltage is not None and component.get("drain_source_voltage_V", 0) < ds_voltage * safety_factor_voltage:
                valid = False
            if current is not None and component.get("current_A", 0) < current * safety_factor_current:
                valid = False
            if on_resistance is not None and component.get("on_resistance_ohms", float('inf')) > on_resistance:
                valid = False
            if power_dissipation is not None and component.get("power_dissipation_W", 0) < power_dissipation:
                valid = False
            if component.get("Stock", 0) <= 100:
                valid = False
            if package and component.get("Package") != package:
                valid = False
            if valid:
                filtered_components.append(component)

        # Select the best component
        if not filtered_components:
            return "No suitable MOSFETs found."
            # return https_fn.Response(
            #     json.dumps({"error": "No suitable MOSFETs found."}),
            #     status=404,
            #     content_type='application/json'
            # )

        best_mosfet = min(filtered_components, key=lambda component: component["Price"][0]["price"] if component["Price"] else float('inf'))

        # Send back a JSON response with the best MOSFET
        return best_mosfet
        # return https_fn.Response(
        #     json.dumps({"Best MOSFET": best_mosfet}),
        #     status=200,
        #     content_type='application/json'
        # )

    except Exception as e:
        return "error" + str(e)
        # return https_fn.Response(
        #     json.dumps({"error": str(e)}),
        #     status=400,
        #     content_type='application/json'
        # )


In [30]:
#

query = {
    "polarity": "N Channel",
    "drain_source_voltage_V": 20,
    "current_A": 20,
    "on_resistance_ohms": 0.1,
    "power_dissipation_W": 23,
    "Package": "PDFN-8(5.2x6.2)"
}

get_mosfet(query, db)

/Users/narayanpowderly/Documents/atopile-workspace/.venv/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:295: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


429


{'Assembly Type': 'SMT AssemblyA PCB assembly fixture is needed to protect and support this part during the assembly process.',
 'value': '100V 63A 78W 10.9mΩ@10V,46A N Channel DFN-8(5x6)  MOSFETs ROHS',
 'type': 'mosfet',
 'uuid': '42142935-f843-4140-b43d-3fef8f87cf78',
 'Description': '100V 63A 78W 10.9mΩ@10V,46A N Channel DFN-8(5x6)  MOSFETs ROHS',
 'Price': [{'quantity': '1+', 'price': 1.326},
  {'quantity': '10+', 'price': 1.1325},
  {'quantity': '30+', 'price': 1.026},
  {'quantity': '100+', 'price': 0.906},
  {'quantity': '500+', 'price': 0.726},
  {'quantity': '1000+', 'price': 0.702}],
 'Stock': 168,
 'CAD Model': 'PCB Footprint or Symbol',
 'Datasheet': 'Download',
 'current_A': 63.0,
 'on_resistance_ohms': 0.0109,
 'Source': 'JLCPCB',
 'Package': 'PDFN-8(5.2x6.2)',
 'drain_source_voltage_V': 100,
 'MFR.Part #': 'BSC109N10NS3G',
 'footprint': {'kicad': 'PDFN-8(5.2x6.2)-42142935-f843-4140-b43d-3fef8f87cf78.kicad_mod'},
 'Manufacturer': 'Infineon Technologies',
 'power_dissipat

In [27]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import uuid

# Reference to the 'mosfets' collection
mosfets_collection = db.collection('mosfets')

# Fetch all documents in the collection
docs = mosfets_collection.stream()

for doc in docs:
    doc_data = doc.to_dict()

    # Move the existing footprint data to 'footprint_data.kicad'
    current_footprint = doc_data.get('footprint').get('kicad')
    doc_data['footprint_data'] = {'kicad': current_footprint}

    # Update the document with new data
    mosfets_collection.document(doc.id).set(doc_data)

    print(f"Updated footprint for {doc_data.get('LCSC Part #', 'Unknown LCSC Part #')}")

print("Database update completed.")


Updated footprint for C105685
Updated footprint for C10647
Updated footprint for C10650
Updated footprint for C10654
Updated footprint for C10657
Updated footprint for C10663
Updated footprint for C10669
Updated footprint for C106992
Updated footprint for C107589
Updated footprint for C108891
Updated footprint for C109445
Updated footprint for C110100
Updated footprint for C110101
Updated footprint for C110158
Updated footprint for C110712
Updated footprint for C110713
Updated footprint for C110715
Updated footprint for C110716
Updated footprint for C110717
Updated footprint for C110718
Updated footprint for C110724
Updated footprint for C110725
Updated footprint for C111356
Updated footprint for C111970
Updated footprint for C111980
Updated footprint for C112722
Updated footprint for C113315
Updated footprint for C115168
Updated footprint for C115827
Updated footprint for C115828
Updated footprint for C115829
Updated footprint for C115833
Updated footprint for C115835
Updated footprin

{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,